## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-04-10-55-59 +0000,nypost,Alleged gunman who shot driver accused of plow...,https://nypost.com/2025/08/04/us-news/alleged-...
1,2025-08-04-10-51-58 +0000,bbc,Deputy First Minister Kate Forbes to stand dow...,https://www.bbc.com/news/articles/c70xejr8drro...
2,2025-08-04-10-49-00 +0000,wsj,Stock Market Today: Dow Futures Rise After Fri...,https://www.wsj.com/livecoverage/stock-market-...
3,2025-08-04-10-46-19 +0000,nypost,Man caught having three marriages at the same ...,https://nypost.com/2025/08/04/us-news/florida-...
4,2025-08-04-10-43-44 +0000,bbc,Farmer accused of feeding women to pigs on tri...,https://www.bbc.com/news/articles/c4gemjql1r5o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
70,trump,35
27,new,15
59,gaza,8
118,house,7
81,russian,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
179,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...,88
77,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...,78
58,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...,72
8,2025-08-04-10-01-12 +0000,nyt,Trump to Appoint New Top Labor Official Within...,https://www.nytimes.com/2025/08/03/us/politics...,64
23,2025-08-04-09-03-44 +0000,bbc,Hundreds of Israeli ex-officials appeal to Tru...,https://www.bbc.com/news/articles/crkznje8nz8o...,62


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
179,88,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...
77,43,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...
149,38,2025-08-03-16-55-11 +0000,nypost,JPMorgan employees gripe about paying for ‘cut...,https://nypost.com/2025/08/03/business/jpm-emp...
113,33,2025-08-03-21-35-51 +0000,nypost,"Israel arming anti-Hamas Gaza clans, including...",https://nypost.com/2025/08/03/world-news/israe...
100,31,2025-08-03-22-56-15 +0000,nypost,Boeing St. Louis defense workers set to strike...,https://nypost.com/2025/08/03/business/boeing-...
185,31,2025-08-03-13-09-24 +0000,cbc,Ukrainian drone attack sets Russian oil depot ...,https://www.cbc.ca/news/world/ukraine-drone-at...
115,27,2025-08-03-21-13-37 +0000,nyt,Third Child in Sailing Camp Dies After Barge H...,https://www.nytimes.com/2025/08/03/us/miami-be...
58,25,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...
9,24,2025-08-04-10-00-57 +0000,wapo,Locals beg for Putin’s help as Russian-occupie...,https://www.washingtonpost.com/world/2025/08/0...
102,24,2025-08-03-22-54-09 +0000,nypost,NYC bodega killer ‘went crazy’ because victim ...,https://nypost.com/2025/08/03/us-news/nyc-bode...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
